In [1]:
import pdfplumber
from statistics import mean


def extract_slides(pdf_path: str,
                   min_avg_len: int = 10,
                   max_lines: int = 20,
                   merge_tol: float = 2.0):
    slides = []
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            txt = page.extract_text() or ""
            lines = [l for l in txt.splitlines() if l.strip()]
            if not lines:                      # page vide
                continue
            if mean(map(len, lines)) < min_avg_len or len(lines) > max_lines:
                continue

            words = page.extract_words(extra_attrs=["size"], use_text_flow=True)
            # regroupe en lignes
            rows, cur, cur_top = [], [], None
            for w in sorted(words, key=lambda w: w["top"]):
                if cur_top is None or abs(w["top"] - cur_top) <= merge_tol:
                    cur.append(w); cur_top = cur_top or w["top"]
                else:
                    rows.append(cur); cur, cur_top = [w], w["top"]
            if cur: rows.append(cur)

            first5 = rows[:5]
            title = (
                " ".join(w["text"] for w in max(first5,
                                                key=lambda r: mean(w["size"] for w in r)))
                if first5 else f"Slide {page.page_number}"
            )

            slides.append({
                "id": f"SL_{page.page_number:03}",   # <─ identifiant slide
                "title": title,
                "content": txt.strip()
            })
    return slides




slides = extract_slides("./volume/slides/cours_1.pdf")

for slide in slides[:10]:
    print(f"Slide number: {slide['id']}")
    print(f"Slide title: {slide['title']}")
    print(slide["content"])
    print("="*100)

Cannot set gray non-stroke color because /'P412' is an invalid float value
Cannot set gray non-stroke color because /'P414' is an invalid float value
Cannot set gray non-stroke color because /'P444' is an invalid float value
Cannot set gray non-stroke color because /'P451' is an invalid float value
Cannot set gray non-stroke color because /'P455' is an invalid float value
Cannot set gray non-stroke color because /'P465' is an invalid float value
Cannot set gray non-stroke color because /'P467' is an invalid float value
Cannot set gray non-stroke color because /'P482' is an invalid float value
Cannot set gray non-stroke color because /'P488' is an invalid float value


Slide number: SL_001
Slide title: 7.1 Architecture du génome
L1SpS: UE 2 Les molécules du vivant
U.E.2 Les molécules du vivant
7. Organisation du génome humain, méthodes en
biotechnologie
7.1 Architecture du génome
humain
Jean Muller
Laboratoire Diagnostic Génétique (HUS)
Laboratoire de Génétique médicale (Inserm U1112)
jeanmuller@unistra.fr
Slide number: SL_002
Slide title: Plan du cours
L1SpS: UE 2 Les molécules du vivant
Plan du cours
• Introduction
• Notions fondamentales
• Le génome humain
• Projet de séquençage
• Architecture globale
• Génome mitochondrial
• Génome nucléaire
• Description des principaux éléments constituants
• Gènes (codants et non-codants), pseudogènes et éléments répétés
• Comparaison aux autres génomes
• Les types de variations du génome humain et leurs conséquences
• La variabilité du génome humain
• Evolution des génomes: notions essentielles, mécanismes
2
Slide number: SL_003
Slide title: Introduction
L1SpS: UE 2 Les molécules du vivant
Introduction
3
Slide

In [2]:
from models import Slides, Content, ContentSection, SectionSlideMapping

formated_slides = []


for slide in slides:
    #-print(type(slide['id']), type(slide['title']), type(slide['content']))
    formated_slides.append(Slides(id=slide['id'], 
                                title=slide['title'],
                                content=slide['content']))

formated_slides

[Slides(id='SL_001', title='7.1 Architecture du génome', content='L1SpS: UE 2 Les molécules du vivant\nU.E.2 Les molécules du vivant\n7. Organisation du génome humain, méthodes en\nbiotechnologie\n7.1 Architecture du génome\nhumain\nJean Muller\nLaboratoire Diagnostic Génétique (HUS)\nLaboratoire de Génétique médicale (Inserm U1112)\njeanmuller@unistra.fr'),
 Slides(id='SL_002', title='Plan du cours', content='L1SpS: UE 2 Les molécules du vivant\nPlan du cours\n• Introduction\n• Notions fondamentales\n• Le génome humain\n• Projet de séquençage\n• Architecture globale\n• Génome mitochondrial\n• Génome nucléaire\n• Description des principaux éléments constituants\n• Gènes (codants et non-codants), pseudogènes et éléments répétés\n• Comparaison aux autres génomes\n• Les types de variations du génome humain et leurs conséquences\n• La variabilité du génome humain\n• Evolution des génomes: notions essentielles, mécanismes\n2'),
 Slides(id='SL_003', title='Introduction', content='L1SpS: UE 2

In [3]:
import pdfplumber
from statistics import mean
from typing import Optional

def extract_pdf_text(pdf_path: str, page_number: Optional[int] = None) -> str:
    """
    Extract text from a PDF file, either from a specific page or the full document.
    
    Args:
        pdf_path: Path to the PDF file
        page_number: Optional page number to extract (1-based). If None, extracts full PDF.
    
    Returns:
        Extracted text as string
    """
    with pdfplumber.open(pdf_path) as pdf:
        if page_number is not None:
            # Adjust for 0-based page indexing
            page_idx = page_number - 1
            if page_idx < 0 or page_idx >= len(pdf.pages):
                raise ValueError(f"Page number {page_number} is out of range")
            
            page = pdf.pages[page_idx]
            text = page.extract_text(layout=True)
            # Remove multiple consecutive newlines
            return '\n'.join(line for line in text.splitlines() if line.strip())
        
        else:
            # Extract full PDF
            all_text = []
            for page in pdf.pages:
                text = page.extract_text(layout=True)
                # Remove multiple consecutive newlines 
                text = '\n'.join(line for line in text.splitlines() if line.strip())
                all_text.append(text)
            return '\n'.join(all_text)

pdf_path = "./volume/slides/cours_1.pdf"
# Example usage:
# full_text = extract_pdf_text(pdf_path)  # Get full PDF
# page_3_text = extract_pdf_text(pdf_path, page_number=3)  # Get page 3 only

# Get full PDF text for now
full_text = extract_pdf_text(pdf_path, page_number=2)
print(full_text)

   L1SpS: UE 2 Les molécules du vivant                                                             
      Plan      du     cours                                                                       
        •  Introduction                                                                            
             •  Notions  fondamentales                                                             
        •  Le génome    humain                                                                     
             •  Projet de séquençage                                                               
             •  Architecture globale                                                               
                  •  Génome    mitochondrial                                                       
                  •  Génome    nucléaire                                                           
             •  Description  des principaux  éléments   constituants                               


In [4]:
from models import Content, ContentSection, SectionSlideMapping
from typing import List, Dict, Optional

In [5]:
import tiktoken
encoder = tiktoken.get_encoding("cl100k_base")  # GPT-4 encoding
text_course_outline = str(full_text)
text_course_content = str(slides)
token_count, token_count_outline = len(encoder.encode(text_course_content)), len(encoder.encode(text_course_outline))
print(token_count, token_count_outline)

13439 221


In [6]:
import dotenv
dotenv.load_dotenv()

import openai

client = openai.OpenAI()

system_prompt = """
You are a course structure analyzer specialized in French academic content.  
Your task is to parse a raw course outline text containing numbered section titles (like "I.", "1.", "1.1.", etc.) or indented bullet points.
and generate a hierarchical JSON structure matching this Pydantic model:

class ContentSection:
    id: str = ""
    title: str
    content: str = ""
    subsections: List[ContentSection]

class Content:
    sections: List[ContentSection]

Requirements:
- Detect all section titles and nest subsections correctly based on numbering.
- Maintain full recursive hierarchy.
- Return valid JSON exactly matching the Content model structure.
- Include every section and subsection with their titles.
- Ignore the content and the id of the sections, you only need to return the outline.
- Return ONLY the JSON, no additional text or explanation.
"""

user_message = full_text  # ton texte brut extrait

response = client.responses.parse(
    model="gpt-5-mini",
    input=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_message}
    ],
    reasoning = {
        "effort": "low",
    },
    text= {"verbosity": "low"},
    
    text_format=Content
)

outline = response.output_parsed

print(outline.print_outline())

Course Outline:
Introduction
  Notions fondamentales
Le génome    humain
  Projet de séquençage
  Architecture globale
    Génome    mitochondrial
    Génome    nucléaire
  Description  des principaux  éléments   constituants
    Gènes  (codants   et non-codants),  pseudogènes     et éléments  répétés
Comparaison    aux  autres génomes
Les types  de variations  du génome    humain   et leurs conséquences
La variabilité du génome   humain
Evolution des  génomes:    notions essentielles,  mécanismes


In [7]:
while len(outline.sections) == 1:
    outline.sections = outline.sections[0].subsections

print(outline.sections[0])

id='' title='Introduction' content=[] subsections=[ContentSection(id='', title='Notions fondamentales', content=[], subsections=[])]


In [8]:
def assign_ids(sec: ContentSection, parent_id: str = "", parent_section: Optional[ContentSection] = None):
    # Determine current section number
    if parent_id == "":
        # Top level section
        sec.id = f"SEC_{len([s for s in outline.sections if s.id.startswith('SEC_')])+ 1}"
    else:
        # Subsection - append next number to parent id
        parent_nums = parent_id.replace("SEC_", "")
        sibling_count = len([s for s in parent_section.subsections if s.id.startswith(f"SEC_{parent_nums}.")])
        sec.id = f"SEC_{parent_nums}.{sibling_count + 1}"
    
    # Recursively assign ids to subsections
    for sub in sec.subsections:
        assign_ids(sub, sec.id, sec)

for s in outline.sections:
    assign_ids(s)


In [9]:
outline_to_send = Content(sections=outline.sections[:])
print(outline_to_send.print_outline())

outline_json = outline_to_send.model_dump_json()

print(outline_json)

Course Outline:
Introduction
  Notions fondamentales
Le génome    humain
  Projet de séquençage
  Architecture globale
    Génome    mitochondrial
    Génome    nucléaire
  Description  des principaux  éléments   constituants
    Gènes  (codants   et non-codants),  pseudogènes     et éléments  répétés
Comparaison    aux  autres génomes
Les types  de variations  du génome    humain   et leurs conséquences
La variabilité du génome   humain
Evolution des  génomes:    notions essentielles,  mécanismes
{"sections":[{"id":"SEC_1","title":"Introduction","content":[],"subsections":[{"id":"SEC_1.1","title":"Notions fondamentales","content":[],"subsections":[]}]},{"id":"SEC_2","title":"Le génome    humain","content":[],"subsections":[{"id":"SEC_2.1","title":"Projet de séquençage","content":[],"subsections":[]},{"id":"SEC_2.2","title":"Architecture globale","content":[],"subsections":[{"id":"SEC_2.2.1","title":"Génome    mitochondrial","content":[],"subsections":[]},{"id":"SEC_2.2.2","title":"Gén

In [10]:
outline

Content(sections=[ContentSection(id='SEC_1', title='Introduction', content=[], subsections=[ContentSection(id='SEC_1.1', title='Notions fondamentales', content=[], subsections=[])]), ContentSection(id='SEC_2', title='Le génome    humain', content=[], subsections=[ContentSection(id='SEC_2.1', title='Projet de séquençage', content=[], subsections=[]), ContentSection(id='SEC_2.2', title='Architecture globale', content=[], subsections=[ContentSection(id='SEC_2.2.1', title='Génome    mitochondrial', content=[], subsections=[]), ContentSection(id='SEC_2.2.2', title='Génome    nucléaire', content=[], subsections=[])]), ContentSection(id='SEC_2.3', title='Description  des principaux  éléments   constituants', content=[], subsections=[ContentSection(id='SEC_2.3.1', title='Gènes  (codants   et non-codants),  pseudogènes     et éléments  répétés', content=[], subsections=[])])]), ContentSection(id='SEC_3', title='Comparaison    aux  autres génomes', content=[], subsections=[]), ContentSection(id=

## Pass 1.bis mapping slide number to section title

In [11]:
def build_mapping_prompt(outline_json: str, slides: List[dict]) -> str:
    """
    Génère le prompt demandant au modèle de produire le mapping section→slides.

    Paramètres
    ----------
    outline_json : str   JSON du plan (chaque section possède un id SEC_***)
    slides       : list  Liste de dicts {"id": "SL_***", "title": ..., "content": ...}

    Retour
    ------
    str : prompt clair et contraignant, prêt pour client.responses.parse
    """
    return f"""
OBJECTIF
Associer chaque section du plan aux diapositives pertinentes.

DONNÉES
1) PLAN_JSON : (structure hiérarchique, chaque section a un identifiant unique "SEC_xxx")


This is the structure of the outline that you will receive:
class ContentSection(BaseModel):
    id: str
    title: str
    content: Optional[str] = ""
    subsections: List["ContentSection"] = Field(default_factory=list)
    
class Content(BaseModel):
    sections: List[ContentSection]

This is the actual outline:

{outline_json}

          


2) SLIDES : (liste ordonnée, chaque slide a un identifiant unique "SL_xxx")
{slides}

FORMAT DE SORTIE — STRICTEMENT OBLIGATOIRE
Un JSON **valide** contenant UNE SEULE clé de premier niveau : "mapping".

This is the pydantic model SectionSlideMapping that you will use to return the mapping:

class MappingItem(BaseModel):
    section_id: str
    slide_ids:  List[str]

class SectionSlideMapping(BaseModel):
    mapping: List[MappingItem]


Exemple de structure attendue :
{{
  "mapping": [
    {{ "section_id": "SEC_1", "slide_ids": ["SL_001", "SL_002"] }},
    {{ "section_id": "SEC_1.1", "slide_ids": ["SL_003", "SL_004"] }}
  ]
}}

CONTRAINTES À RESPECTER IMPÉRATIVEMENT
- Chaque "section_id" présent dans PLAN_JSON doit apparaître **exactement une fois**.
- Chaque "slide_id" présent dans SLIDES doit apparaître **au moins une fois** (aucune slide orpheline).
- "slide_ids" est un tableau d'identifiants ; l’ordre interne n’a pas d’importance.
- N’utilise **que** des identifiants existants ; n’invente pas de clés ni de champs.
- AUCUNE explication, commentaire ou clé supplémentaire : la sortie doit être **uniquement** le JSON demandé.
""".strip()


mapping_rsp = client.responses.parse(
    model="gpt-5-mini",
    input=[{"role": "user", "content": build_mapping_prompt(outline_json, slides)}],
    text_format=SectionSlideMapping,
    reasoning={"effort": "high"},
    text={"verbosity": "low"},  # ← schéma fixe
)
slides_by_section: Dict[str, List[str]] = {
    item.section_id: item.slide_ids for item in mapping_rsp.output_parsed.mapping
}

In [13]:
section_mapping = mapping_rsp.output_parsed

print(section_mapping.visualize_mapping(outline))

Section-to-Slides Mapping:

[Root] SEC_1
Title: Introduction
Slides: 3 slide(s)
Slide IDs: SL_002, SL_003, SL_012

  [Level 1] SEC_1.1
  Title: Notions fondamentales
  Slides: 8 slide(s)
  Slide IDs: SL_004, SL_005, SL_006, SL_007, SL_008, SL_009, SL_010, SL_011

[Root] SEC_2
Title: Le génome    humain
Slides: 7 slide(s)
Slide IDs: SL_013, SL_014, SL_015, SL_016, SL_017, SL_030, SL_067

  [Level 1] SEC_2.1
  Title: Projet de séquençage
  Slides: 5 slide(s)
  Slide IDs: SL_018, SL_019, SL_020, SL_021, SL_022

  [Level 1] SEC_2.2
  Title: Architecture globale
  Slides: 4 slide(s)
  Slide IDs: SL_001, SL_024, SL_025, SL_028

    [Level 2] SEC_2.2.1
    Title: Génome    mitochondrial
    Slides: 3 slide(s)
    Slide IDs: SL_031, SL_032, SL_033

    [Level 2] SEC_2.2.2
    Title: Génome    nucléaire
    Slides: 2 slide(s)
    Slide IDs: SL_034, SL_035

  [Level 1] SEC_2.3
  Title: Description  des principaux  éléments   constituants
  Slides: 1 slide(s)
  Slide IDs: SL_036

    [Level 2] SE

### New way to build the Content object

In [16]:
# Just call the method on the existing outline:
outline.enrich_with_slides(formated_slides, section_mapping)
outline

Content(sections=[ContentSection(id='SEC_1', title='Introduction', content=['L1SpS: UE 2 Les molécules du vivant\nPlan du cours\n• Introduction\n• Notions fondamentales\n• Le génome humain\n• Projet de séquençage\n• Architecture globale\n• Génome mitochondrial\n• Génome nucléaire\n• Description des principaux éléments constituants\n• Gènes (codants et non-codants), pseudogènes et éléments répétés\n• Comparaison aux autres génomes\n• Les types de variations du génome humain et leurs conséquences\n• La variabilité du génome humain\n• Evolution des génomes: notions essentielles, mécanismes\n2', 'L1SpS: UE 2 Les molécules du vivant\nIntroduction\n3', 'L1SpS: UE 2 Les molécules du vivant\nRéférences\n• Le génome est une des références d’un organisme\n…\nméthylome\nprotéome\ntranscriptome\ngénome\n12'], subsections=[ContentSection(id='SEC_1.1', title='Notions fondamentales', content=['L1SpS: UE 2 Les molécules du vivant\nL’arbre de la vie\nNotions essentielles:\n• Evolution\n• Temps\n• Adapt

In [14]:
# Just call the method on the existing outline:
outline_to_send.enrich_with_slides(formated_slides, section_mapping)
outline_to_send

Content(sections=[ContentSection(id='SEC_1', title='Introduction', content=['L1SpS: UE 2 Les molécules du vivant\nPlan du cours\n• Introduction\n• Notions fondamentales\n• Le génome humain\n• Projet de séquençage\n• Architecture globale\n• Génome mitochondrial\n• Génome nucléaire\n• Description des principaux éléments constituants\n• Gènes (codants et non-codants), pseudogènes et éléments répétés\n• Comparaison aux autres génomes\n• Les types de variations du génome humain et leurs conséquences\n• La variabilité du génome humain\n• Evolution des génomes: notions essentielles, mécanismes\n2', 'L1SpS: UE 2 Les molécules du vivant\nIntroduction\n3', 'L1SpS: UE 2 Les molécules du vivant\nRéférences\n• Le génome est une des références d’un organisme\n…\nméthylome\nprotéome\ntranscriptome\ngénome\n12'], subsections=[ContentSection(id='SEC_1.1', title='Notions fondamentales', content=['L1SpS: UE 2 Les molécules du vivant\nL’arbre de la vie\nNotions essentielles:\n• Evolution\n• Temps\n• Adapt

In [20]:
new_content_json = outline_to_send.model_dump_json()

new_content_json

'{"sections":[{"id":"SEC_1","title":"Introduction","content":["L1SpS: UE 2 Les molécules du vivant\\nPlan du cours\\n• Introduction\\n• Notions fondamentales\\n• Le génome humain\\n• Projet de séquençage\\n• Architecture globale\\n• Génome mitochondrial\\n• Génome nucléaire\\n• Description des principaux éléments constituants\\n• Gènes (codants et non-codants), pseudogènes et éléments répétés\\n• Comparaison aux autres génomes\\n• Les types de variations du génome humain et leurs conséquences\\n• La variabilité du génome humain\\n• Evolution des génomes: notions essentielles, mécanismes\\n2","L1SpS: UE 2 Les molécules du vivant\\nIntroduction\\n3","L1SpS: UE 2 Les molécules du vivant\\nRéférences\\n• Le génome est une des références d’un organisme\\n…\\nméthylome\\nprotéome\\ntranscriptome\\ngénome\\n12"],"subsections":[{"id":"SEC_1.1","title":"Notions fondamentales","content":["L1SpS: UE 2 Les molécules du vivant\\nL’arbre de la vie\\nNotions essentielles:\\n• Evolution\\n• Temps\\n• 

In [21]:
print(outline_to_send.print_outline())

Course Outline:
Introduction
  Notions fondamentales
Le génome    humain
  Projet de séquençage
  Architecture globale
    Génome    mitochondrial
    Génome    nucléaire
  Description  des principaux  éléments   constituants
    Gènes  (codants   et non-codants),  pseudogènes     et éléments  répétés
Comparaison    aux  autres génomes
Les types  de variations  du génome    humain   et leurs conséquences
La variabilité du génome   humain
Evolution des  génomes:    notions essentielles,  mécanismes


In [22]:
for section in outline_to_send.sections:
    print("--------------------------------")
    print(section.title)
    print(section.get_token_count())

--------------------------------
Introduction
1232
--------------------------------
Le génome    humain
5455
--------------------------------
Comparaison    aux  autres génomes
432
--------------------------------
Les types  de variations  du génome    humain   et leurs conséquences
3053
--------------------------------
La variabilité du génome   humain
187
--------------------------------
Evolution des  génomes:    notions essentielles,  mécanismes
837


In [24]:
course_with_slides = outline.model_copy()

# Volumen of tokens

In [25]:
import tiktoken
encoder = tiktoken.get_encoding("cl100k_base")  # GPT-4 encoding
text_course_with_slides = str(course_with_slides)
token_count = len(encoder.encode(text_course_with_slides))
print(token_count)

25031


In [26]:
content_to_send = course_with_slides
content_json_str = content_to_send.model_dump_json()
content_json_str

'{"sections":[{"id":"SEC_1","title":"Introduction","content":["L1SpS: UE 2 Les molécules du vivant\\nPlan du cours\\n• Introduction\\n• Notions fondamentales\\n• Le génome humain\\n• Projet de séquençage\\n• Architecture globale\\n• Génome mitochondrial\\n• Génome nucléaire\\n• Description des principaux éléments constituants\\n• Gènes (codants et non-codants), pseudogènes et éléments répétés\\n• Comparaison aux autres génomes\\n• Les types de variations du génome humain et leurs conséquences\\n• La variabilité du génome humain\\n• Evolution des génomes: notions essentielles, mécanismes\\n2","L1SpS: UE 2 Les molécules du vivant\\nIntroduction\\n3","L1SpS: UE 2 Les molécules du vivant\\nRéférences\\n• Le génome est une des références d’un organisme\\n…\\nméthylome\\nprotéome\\ntranscriptome\\ngénome\\n12"],"subsections":[{"id":"SEC_1.1","title":"Notions fondamentales","content":["L1SpS: UE 2 Les molécules du vivant\\nL’arbre de la vie\\nNotions essentielles:\\n• Evolution\\n• Temps\\n• 

In [27]:
encoder = tiktoken.get_encoding("cl100k_base")  # GPT-4 encoding
text_course_with_slides = str(content_json_str)
token_count = len(encoder.encode(text_course_with_slides))
print(token_count)

24591


In [28]:
def build_system_prompt() -> str:
    """
    Prompt système définissant le rôle et les règles
    """
    return """Tu es un expert en rédaction pédagogique. Ta tâche est de transformer du contenu brut de slides en paragraphes de cours structurés.

Règles :
- Traite TOUTES les sections du JSON (ne t'arrête pas à la première)
- Pour chaque section, synthétise tous les éléments content[] en 2-10 paragraphes cohérents
- Chaque paragraphe = maximum 10 phrases liées au titre de la section
- Conserve la structure JSON exacte (mêmes IDs, titres, ordre)
- Style académique fluide"""


def build_user_prompt(content_json: str) -> str:
    """
    Prompt utilisateur avec le contenu à traiter
    """
    return content_json


def build_assistant_prompt() -> str:
    """
    Prompt assistant avec exemple d'input/output
    """
    return """Voici un exemple de transformation attendue :

INPUT EXEMPLE:
```json
{
  "sections": [
    {
      "id": "SEC_1",
      "title": "Introduction aux Algorithmes",
      "content": [
        "CS101: Algorithmique\\nAlgorithmes\\n• Définition: séquence d'instructions\\n• Résolution problèmes\\n• Entrée → Traitement → Sortie\\n• Propriétés essentielles\\n1",
        "CS101: Algorithmique\\nCaractéristiques\\n• Fini (nombre étapes)\\n• Non-ambigu\\n• Déterministe\\n• Efficacité importante\\n• Complexité temporelle/spatiale\\n2",
        "CS101: Algorithmique\\nTypes d'algorithmes\\n• Itératifs vs Récursifs\\n• Diviser pour régner\\n• Gloutons (greedy)\\n• Programmation dynamique\\nExemples concrets\\n3"
      ],
      "subsections": [
        {
          "id": "SEC_1.1",
          "title": "Notation Big-O",
          "content": [
            "CS101: Algorithmique\\nComplexité algorithmique\\n• Notation O(n)\\n• Mesure efficacité\\n• Pire cas considéré\\n• Croissance asymptotique\\n4",
            "CS101: Algorithmique\\nExemples complexités\\n• O(1) - constant\\n• O(log n) - logarithmique\\n• O(n) - linéaire\\n• O(n²) - quadratique\\n• O(2^n) - exponentielle\\n5"
          ],
          "subsections": []
        }
      ]
    },
    {
      "id": "SEC_2", 
      "title": "Structures de Données",
      "content": [
        "CS101: Algorithmique\\nStructures fondamentales\\n• Tableaux (arrays)\\n• Listes chaînées\\n• Piles (stacks)\\n• Files (queues)\\n• Organisation mémoire\\n6",
        "CS101: Algorithmique\\nOpérations de base\\n• Insertion / Suppression\\n• Recherche / Accès\\n• Parcours / Tri\\n• Complexité variable\\n• Trade-offs temps/espace\\n7"
      ],
      "subsections": []
    }
  ]
}
```

OUTPUT EXEMPLE:
```json
{
  "sections": [
    {
      "id": "SEC_1",
      "title": "Introduction aux Algorithmes", 
      "content": [
        "Un algorithme constitue une séquence finie et ordonnée d'instructions permettant de résoudre un problème spécifique ou d'accomplir une tâche donnée. Cette approche systématique transforme des données d'entrée en résultats de sortie par l'application d'un processus de traitement déterministe et reproductible.",
        "Les algorithmes présentent des caractéristiques fondamentales qui garantissent leur validité : ils doivent être finis dans le nombre d'étapes, non-ambigus dans leur interprétation, et déterministes dans leur exécution. L'efficacité algorithmique, mesurée par les complexités temporelle et spatiale, constitue un critère essentiel d'évaluation de leur performance.",
        "La classification des algorithmes distingue plusieurs approches méthodologiques selon leur stratégie de résolution : les algorithmes itératifs et récursifs, les méthodes diviser-pour-régner, les algorithmes gloutons, et la programmation dynamique. Chaque paradigme offre des avantages spécifiques selon la nature du problème à traiter."
      ],
      "subsections": [
        {
          "id": "SEC_1.1",
          "title": "Notation Big-O",
          "content": [
            "La notation Big-O fournit un outil mathématique pour caractériser la complexité algorithmique en décrivant la croissance asymptotique du temps d'exécution ou de l'espace mémoire requis. Cette mesure considère le comportement dans le pire cas et permet de comparer objectivement l'efficacité de différents algorithmes.",
            "Les classes de complexité courantes s'organisent selon une hiérarchie croissante : O(1) pour les opérations à temps constant, O(log n) pour les algorithmes logarithmiques, O(n) pour les traitements linéaires, O(n²) pour les approches quadratiques, et O(2^n) pour les solutions exponentielles. Cette classification guide le choix algorithmique selon les contraintes de performance requises."
          ],
          "subsections": []
        }
      ]
    },
    {
      "id": "SEC_2",
      "title": "Structures de Données",
      "content": [
        "Les structures de données fondamentales organisent et stockent l'information selon des patterns spécifiques adaptés aux besoins algorithmiques : les tableaux offrent un accès direct par index, les listes chaînées permettent une allocation dynamique, tandis que les piles et files implémentent des politiques d'accès particulières. L'organisation mémoire influence directement les performances d'accès et de manipulation.",
        "Les opérations de base sur ces structures - insertion, suppression, recherche, accès, parcours et tri - présentent des complexités variables selon l'implémentation choisie. Ces trade-offs entre temps d'exécution et espace mémoire orientent la sélection de la structure la plus appropriée selon les contraintes spécifiques de chaque application."
      ],
      "subsections": []
    }
  ]
}
```

Maintenant traite le JSON fourni en suivant ce modèle :"""


def build_prompt_fill_content(content_json: str) -> str:
    """
    Version legacy - maintenant utilise les 3 fonctions séparées ci-dessus
    """
    return f"""
**SYSTEM:**
{build_system_prompt()}

**USER:**
{build_user_prompt(content_json)}

**ASSISTANT:**
{build_assistant_prompt()}
""".strip()


prompt_to_send = build_prompt_fill_content(content_json_str)

response = client.responses.parse(
    model="gpt-5-mini",
    input=[
        {
            "role": "system",
            "content": "Tu es un Professeur qui remplit le contenu des sections du plan de cours à partir du contenu des slides.",
        },
        {"role": "user", "content": prompt_to_send},
    ],
    text_format=Content,
    reasoning={"effort": "high"},  # for deep synthesis
    text={"verbosity": "high"},  # for complete, detailed course material
)


print(response.output_parsed)

sections=[ContentSection(id='SEC_1', title='Introduction', content=["Ce module présente une vue d'ensemble des molécules du vivant en prenant comme fil conducteur le génome humain. Le plan du cours énonce les thèmes abordés : notions fondamentales, architecture du génome, projet de séquençage, génomes mitochondrial et nucléaire, éléments constitutifs (gènes codants et non codants, pseudogènes, éléments répétés), comparaison avec d'autres génomes, types de variations, variabilité et mécanismes évolutifs.", "Le génome est présenté comme une des références centrales d'un organisme, au même titre que le protéome, le transcriptome ou le méthylome. Il constitue une base informationnelle et structurale essentielle pour comprendre l'expression, la régulation et l'évolution des caractères biologiques."], subsections=[ContentSection(id='SEC_1.1', title='Notions fondamentales', content=["L'arbre de la vie rappelle que les êtres vivants partagent une histoire évolutive commune et que des notions t

In [31]:
course = response.output_parsed
print(course.print_outline())

Course Outline:
Introduction
  Notions fondamentales
Le génome    humain
  Projet de séquençage
  Architecture globale
    Génome    mitochondrial
    Génome    nucléaire
  Description  des principaux  éléments   constituants
    Gènes  (codants   et non-codants),  pseudogènes     et éléments  répétés
Comparaison    aux  autres génomes
Les types  de variations  du génome    humain   et leurs conséquences
La variabilité du génome   humain
Evolution des  génomes:    notions essentielles,  mécanismes


In [33]:
print(course.print_content())

Course Content:

[Root] ID: SEC_1
Title: Introduction
Content:
  [1] Ce module présente une vue d'ensemble des molécules du vivant en prenant
      comme fil conducteur le génome humain. Le plan du cours énonce les thèmes
      abordés : notions fondamentales, architecture du génome, projet de
      séquençage, génomes mitochondrial et nucléaire, éléments constitutifs
      (gènes codants et non codants, pseudogènes, éléments répétés), comparaison
      avec d'autres génomes, types de variations, variabilité et mécanismes
      évolutifs.
  [2] Le génome est présenté comme une des références centrales d'un organisme,
      au même titre que le protéome, le transcriptome ou le méthylome. Il
      constitue une base informationnelle et structurale essentielle pour
      comprendre l'expression, la régulation et l'évolution des caractères
      biologiques.

  [Level 1] ID: SEC_1.1
  Title: Notions fondamentales
  Content:
    [1] L'arbre de la vie rappelle que les êtres vivants partagent

In [23]:
encoder = tiktoken.get_encoding("cl100k_base")  # GPT-4 encoding
text_course_with_slides = str(course)
token_count = len(encoder.encode(text_course_with_slides))
print(token_count)

3992
